<a href="https://colab.research.google.com/github/yurabuturlia/coursera2/blob/main/part_abcd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A. Build a baseline model

Use the Keras library to build a neural network with the following:

- One hidden layer of 10 nodes, and a ReLU activation function

- Use the adam optimizer and the mean squared error  as the loss function.

1. Randomly split the data into a training and test sets by holding 30% of the data for testing. You can use the train_test_splithelper function from Scikit-learn.

2. Train the model on the training data using 50 epochs.

3. Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the mean_squared_error function from Scikit-learn.

4. Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.

5. Report the mean and the standard deviation of the mean squared errors.


Download and Clean Dataset


In [1]:
import pandas as pd
import numpy as np

Let's download the data and read it into a <em>pandas</em> dataframe.


In [2]:
concrete_data = pd.read_csv('https://cocl.us/concrete_data')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
concrete_data.shape

(1030, 9)

Let's check the dataset for any missing values.


In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.


#### Split data into predictors and target


The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.


In [6]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column
n_cols = predictors.shape[1] # number of predictors
print(n_cols)

8


Let's do a quick sanity check of the predictors and the target dataframes.


In [7]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [8]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.


In [9]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [10]:
import keras

In [11]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

Build a Neural Network


Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.


In [12]:
# define regression model
def regression_model(layers_cnt):
    # create model
    nodes=10    
    model = Sequential()
    model.add(Dense(nodes, activation='relu', input_shape=(n_cols,))) 
    if layers_cnt>0:
      for i in range(layers_cnt):
        model.add(Dense(nodes, activation='relu'))
    model.add(Dense(1))    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# **Let's define function to perform:**
# 1.Randomly split the data into a training and test sets by holding 30% of the data for testing. You can use the train_test_splithelper function from Scikit-learn.
# 2.Train the model on the training data using 50 epochs.
# 3. Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the mean_squared_error function from Scikit-learn.



In [13]:
def build_model(x,y,epochs_number,additional_layers_number=0):
  model = regression_model(additional_layers_number)
  #1.Randomly split the data into a training and test sets by holding 30% of the data for testing. You can use the train_test_splithelper function from Scikit-learn.
  X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3,random_state=10)
  #2.Train the model on the training data using 50 epochs.
  model.fit(X_train, y_train, epochs=epochs_number,verbose=0)
  #3.Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the mean_squared_error function from Scikit-learn.
  y_pred = model.predict(X_test)
  return mean_squared_error(y_test, y_pred)

Call function

In [14]:
print('MSE  is %.6f' % build_model(predictors, target,50))

MSE  is 733.131268


# 4. Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.

In [15]:
mse_list = []
for i in range(50):
  mse= build_model(predictors, target,50)
  mse_list.append(mse)
print('mse: \n',mse_list)

mse: 
 [213.64300716318843, 581.6123903043884, 1281.6419114007888, 431.87875224575373, 145.0170582765223, 110.9002743036491, 882.5760547545033, 311.142777634627, 200.74984554243272, 184.90535552014478, 276.78132165005786, 287.49691289865024, 143.99054102021157, 90.3022648134334, 360.3233214813018, 103.05248306658414, 1189.5598370717814, 168.79204199470618, 352.1430407117613, 84.89404568276788, 123.14735893107948, 424.7188638233603, 127.12904807240696, 109.67399883976418, 78.47426468324213, 671.9422806219727, 120.58571507166982, 239.2081634471013, 124.1967609022127, 102.9281283155702, 113.19263021230333, 420.00698128951177, 162.08364657073466, 1202.6818524452904, 1570.7818944524638, 191.91468432109545, 173.29526971631324, 131.83596912392477, 1557.921895968999, 104.72374012666982, 106.83726154976394, 195.45979646458088, 102.91442916166848, 149.67778927723896, 771.7268618628664, 753.4425397464897, 132.00053244649757, 121.7272850842238, 174.2583434041836, 137.880197004512]


# 5. Report the mean and the standard deviation of the mean squared errors.

In [16]:
mean = np.mean(mse_list)
stddev = np.std(mse_list)
print("mean=%.6f, std dev=%.6f" %(mean, stddev))

mean=355.955428, std dev=387.892575


# B. Normalize the data

Repeat Part A but use a normalized version of the data. Recall that one way to normalize the data is by subtracting the mean from the individual predictors and dividing by the standard deviation.

How does the mean of the mean squared errors compare to that from Step A?

In [17]:
print('MSE  is %.6f' % build_model(predictors_norm, target,50))

MSE  is 383.579674


In [18]:
mse_list_norm = []
for i in range(50):
  mse_norm= build_model(predictors_norm, target,50)
  mse_list_norm.append(mse_norm)
print('mse: \n',mse_list_norm)

mse: 
 [315.7374669909772, 330.2334380972114, 268.23537720356416, 324.35157157693334, 312.20242340878497, 301.80557609135263, 306.7542456254427, 297.1597234615703, 406.36246248149865, 303.50205983891567, 418.19782992568827, 326.70836858749743, 242.04897399085934, 301.45126793866945, 307.1255726366676, 338.39682378014453, 457.01920343515934, 350.4955468776828, 372.2223355384154, 546.4698922344605, 269.19367441397094, 387.1894296817021, 302.2485510860888, 274.6508145178524, 266.2893419167829, 370.4681456402292, 242.464326206953, 369.6188752129098, 400.41693543420297, 257.4480522415384, 368.36672970165927, 584.0417489348042, 269.5012108824701, 340.7694731763006, 344.1498765706615, 496.1825007231984, 407.9092648398352, 259.361957251173, 711.7727193708185, 348.778610198033, 297.1852839794914, 410.3217383916964, 423.86954166943104, 341.9753757817055, 358.9185906675231, 306.0209168191715, 297.7932245692799, 235.21990598774786, 261.6459415558553, 396.1435388374312]


In [19]:
# Evaluate mean and std
mean_norm = np.mean(mse_list_norm)
stddev_norm = np.std(mse_list_norm)
print("mean norm = %.6f, std dev norm = %.6f" %(mean_norm, stddev_norm))
print("mean = %.6f, std dev = %.6f" %(mean, stddev))


mean norm = 348.527929, std dev norm = 90.106572
mean = 355.955428, std dev = 387.892575


# C. Increate the number of epochs

Repeat Part B but use 100 epochs this time for training.

How does the mean of the mean squared errors compare to that from Step B?

In [20]:
print('MSE  is %.6f' % build_model(predictors_norm, target,100))

MSE  is 153.052571


In [21]:
mse_list_norm_c = []
for i in range(50):
  mse_norm_c= build_model(predictors_norm, target,100)
  mse_list_norm_c.append(mse_norm_c)
print('mse norm 100ep: \n',mse_list_norm_c)

mse norm 100ep: 
 [141.70687837543312, 151.2419310354359, 144.76831215365377, 154.0273364240737, 138.60624723883979, 150.4871925724421, 158.14159847766177, 144.25267970792774, 150.76868345680103, 133.3328319606607, 144.75345618948057, 148.4728941384798, 149.20819114732143, 163.76274938078174, 149.43932116300735, 148.9063506464211, 144.26753394254172, 142.3122669566092, 149.1628177287506, 156.66486363931355, 216.43417177474254, 153.80972780624808, 140.71732526104094, 142.47052983777934, 156.8403905422841, 146.34674792184953, 161.32289335758128, 138.60425331310088, 154.69849057795363, 158.23489097304008, 149.62052915983404, 135.37195514542822, 163.77314406369632, 152.37445863643873, 140.84037085966668, 153.0411438185648, 137.46469454064393, 152.80163848825285, 143.31323152336162, 162.73347728329762, 141.0292665255572, 187.43487363147574, 140.7314362325246, 146.5428388081661, 167.96807275929925, 172.05007233116982, 193.7240350001817, 153.4095637458692, 147.93649763282355, 179.556835434778

In [22]:
# Evaluate mean and std
mean_norm_c = np.mean(mse_list_norm_c)
stddev_norm_c = np.std(mse_list_norm_c)
print("mean norm = %.6f, std dev norm = %.6f" %(mean_norm, stddev_norm))
print("mean norm 100 ep = %.6f, std dev 100 ep = %.6f" %(mean_norm_c, stddev_norm_c))

mean norm = 348.527929, std dev norm = 90.106572
mean norm 100 ep = 153.109634, std dev 100 ep = 15.192186


# D. Increase the number of hidden layers 

Repeat part B but use a neural network with the following instead:

- Three hidden layers, each of 10 nodes and ReLU activation function.

How does the mean of the mean squared errors compare to that from Step B?

In [23]:
print('MSE norm 3 layers  is %.6f' % build_model(predictors_norm, target,50,2))

MSE norm 3 layers  is 129.213612


In [ ]:
mse_list_norm_d = []
additional_layers=2
epochs=50
for i in range(50):
  mse_norm_d= build_model(predictors_norm, target,epochs,additional_layers)
  mse_list_norm_d.append(mse_norm_d)
print('mse norm 3 layers: \n',mse_list_norm_d)

In [ ]:
# Evaluate mean and std
mean_norm_d = np.mean(mse_list_norm_d)
stddev_norm_d = np.std(mse_list_norm_d)
print("mean norm = %.6f, std dev norm = %.6f" %(mean_norm, stddev_norm))
print("mean norm 3 layers = %.6f, std dev 3 layers = %.6f" %(mean_norm_d, stddev_norm_d))